In [1]:
from etl.spark_initializer import SparkInitializer
from etl.extractor.extractor import DataFrameExtractor
from etl.tranform.transform import CustomerDataTransformer,PurchaseHistoryDataTransformer,ProductDataTransformer
from etl.validator_data import ValidatorImpl
from etl.load.loader import LoadDataToMysql
import os
from etl.sql_executor import MysqlExecutor
# Set SPARK_CLASSPATH to include JARs
os.environ["SPARK_CLASSPATH"] = os.path.abspath("jars/hadoop-aws-3.3.4.jar") + ";" + os.path.abspath("jars/aws-java-sdk-bundle-1.12.262.jar")

# Reset and initialize Spark session
SparkInitializer._spark = None
spark = SparkInitializer.get_spark()

In [32]:
# Debugging Spark Configuration
print("Spark Configuration:")
for key, value in spark.sparkContext.getConf().getAll():
    print(f"{key}: {value}")

# Debugging AWS Credentials
from etl.config.aws_config import AWSConfig
aws_config = AWSConfig()
print(f"AWS Access Key: {aws_config.access_key}")
print(f"AWS Secret Key: {'***' if aws_config.secret_key else None}")
print(f"S3 Bucket: {aws_config.bucket_name}")

Spark Configuration:
spark.driver.port: 62728
spark.driver.extraJavaOptions: -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false
spark.hadoop.fs.s3a.secret.key: mbEHEER/07OjGFAE3JY6l9JpEaXP3aDaQIT

**Extractor**

In [2]:
# Step 1: Extract data from S3 with error handling
try:
    fas_purchase_df = (DataFrameExtractor()
                        .extract('csv', 'raw/fashion_purchase_history.csv'))
    print("Successfully loaded fashion purchase data")
    fas_purchase_df.show(3)
    
    customer_df = (DataFrameExtractor()
                        .extract('csv', 'raw/customer.csv'))
    print("Successfully loaded customer data")
    customer_df.show(3)
    
    product_df = (DataFrameExtractor()
                        .extract('csv', 'raw/products.csv'))
    print("Successfully loaded product data")
    product_df.show(3)
    
except Exception as e:
    print(f"Error loading data from S3: {str(e)}")
    import traceback
    traceback.print_exc()

Successfully loaded fashion purchase data
+-----------+--------------+---------------+--------+-------------+-------------+--------------+
|customer_id|item_purchased|purchase_amount|quantity|date_purchase|review_rating|payment_method|
+-----------+--------------+---------------+--------+-------------+-------------+--------------+
|       4115|         Tunic|         2456.0|       3|   2023-07-11|          2.0|   Credit Card|
|       4019|      Tank Top|         2102.0|       2|   2023-03-23|          4.1|          Cash|
|       4097|      Leggings|         3126.0|       4|   2023-03-15|          3.2|          Cash|
+-----------+--------------+---------------+--------+-------------+-------------+--------------+
only showing top 3 rows

Successfully loaded customer data
+-----------+----------+---------+------+-------------------+--------------------+------------+-------------------+------------+-----------+
|customer_id|first_name|last_name|gender|      date_of_birth|               ema

**Transfrom**

In [4]:
# kiểm tra kiểu dữ liệu
print('\033[1m' +'Data type:')
customer_df.printSchema()
fas_purchase_df.printSchema()
product_df.printSchema()

# Check missing value
print('\033[1m' + 'Null:')
ValidatorImpl().check_null_values(customer_df)
ValidatorImpl().check_null_values(fas_purchase_df)
ValidatorImpl().check_null_values(product_df)

# Check duplicate value
print('\033[1m' +'Duplicate values:')
ValidatorImpl().check_duplicate_records(customer_df,['customer_id', 'first_name', 'last_name', 'gender', 'date_of_birth'])
ValidatorImpl().check_duplicate_records(fas_purchase_df,['customer_id', 'item_purchased', 'date_purchase'])
ValidatorImpl().check_duplicate_records(product_df,['item', 'category'])

Data type:
root
 |-- customer_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- date_of_birth: timestamp (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- signup_date: timestamp (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)

root
 |-- customer_id: integer (nullable = true)
 |-- item_purchased: string (nullable = true)
 |-- purchase_amount: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- date_purchase: date (nullable = true)
 |-- review_rating: double (nullable = true)
 |-- payment_method: string (nullable = true)

root
 |-- item: string (nullable = true)
 |-- category: string (nullable = true)

Null:
+-----------+----------+---------+------+-------------+-----+------------+-----------+-------+----+
|customer_id|first_name|last_name|gender|date_of_birth|email|p

In [5]:
'''
Change data type:
    - date_of_birth: TimeStamp -> Date
    - date_purchase: Date -> Timestamp
    - review_rating: Double -> Floatx
'''
cleand_customer_df = CustomerDataTransformer().transform(customer_df)

cleand_fas_purchase_df = PurchaseHistoryDataTransformer().transform(fas_purchase_df)

cleand_product_df = ProductDataTransformer().transform(product_df)

In [6]:
# Check data type
cleand_customer_df.printSchema()
cleand_fas_purchase_df.printSchema()
cleand_product_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- first_name: string (nullable = false)
 |-- last_name: string (nullable = false)
 |-- gender: string (nullable = false)
 |-- date_of_birth: date (nullable = true)
 |-- email: string (nullable = false)
 |-- phone_number: string (nullable = false)
 |-- signup_date: timestamp (nullable = true)
 |-- address: string (nullable = false)
 |-- city: string (nullable = false)

root
 |-- customer_id: integer (nullable = true)
 |-- item_purchased: string (nullable = true)
 |-- purchase_amount: double (nullable = false)
 |-- quantity: integer (nullable = true)
 |-- date_purchase: timestamp (nullable = true)
 |-- review_rating: float (nullable = false)
 |-- payment_method: string (nullable = true)

root
 |-- item: string (nullable = true)
 |-- category: string (nullable = false)



In [7]:
# Check duplicate value
ValidatorImpl().check_duplicate_records(cleand_customer_df,['customer_id', 'first_name', 'last_name', 'gender', 'date_of_birth'])
ValidatorImpl().check_duplicate_records(cleand_fas_purchase_df,['customer_id', 'item_purchased', 'date_purchase'])
ValidatorImpl().check_duplicate_records(cleand_product_df,['item', 'category'])


+-----------+----------+---------+------+-------------+---------------+
|customer_id|first_name|last_name|gender|date_of_birth|duplicate_count|
+-----------+----------+---------+------+-------------+---------------+
+-----------+----------+---------+------+-------------+---------------+

+-----------+--------------+-------------+---------------+
|customer_id|item_purchased|date_purchase|duplicate_count|
+-----------+--------------+-------------+---------------+
+-----------+--------------+-------------+---------------+

+----+--------+---------------+
|item|category|duplicate_count|
+----+--------+---------------+
+----+--------+---------------+



**Load**

In [8]:
cleand_customer_df.show(3)
cleand_fas_purchase_df.show(3)
cleand_product_df.show(3)

+-----------+----------+---------+------+-------------+--------------------+------------+-------------------+-------------+-------+
|customer_id|first_name|last_name|gender|date_of_birth|               email|phone_number|        signup_date|      address|   city|
+-----------+----------+---------+------+-------------+--------------------+------------+-------------------+-------------+-------+
|       3964|      Alex|  Johnson|Female|   1972-07-14|robert.williams87...|291-285-3647|2017-09-17 14:04:33|  7255 Oak St|Phoenix|
|       3970|      Alex|    Jones|Female|   1981-10-08|jane.williams34@m...|869-802-7565|2015-07-10 17:08:42|5623 Maple St|Chicago|
|       3995|      Alex|    Jones|  Male|   1997-02-05|sarah.martinez133...|924-800-7391|2019-09-13 14:21:15| 5180 Main St|Houston|
+-----------+----------+---------+------+-------------+--------------------+------------+-------------------+-------------+-------+
only showing top 3 rows

+-----------+--------------+---------------+-------

In [9]:
## Create customer, product and puchase_history_stagging table 
sql_statements =[
    """
    CREATE TABLE customer(
        customer_id INT PRIMARY KEY AUTO_INCREMENT,
        first_name VARCHAR(20),
        last_name VARCHAR(20),
        gender VARCHAR(10),
        date_of_birth DATE,
        email VARCHAR(50),
        phone_number VARCHAR(12),
        signup_date TIMESTAMP, 
        address VARCHAR(255),
        city VARCHAR(50)    
    )
    """,
    """
    CREATE TABLE product (
        item VARCHAR(50) PRIMARY KEY,
        category VARCHAR(50)
    )
    """,
    """
    CREATE TABLE purchase_history_stagging (
        purchase_id INT PRIMARY KEY AUTO_INCREMENT,
        customer_id INT,
        item_purchased VARCHAR(50),
        purchase_amount DOUBLE,
        quantity INT,
        date_purchase TIMESTAMP,
        review_rating FLOAT,
        payment_method VARCHAR(15)
       )   
    """    
]
message = '✅ Creation table success!' 
error_message = '❌ Error:'
for statement in sql_statements:
    try:
        MysqlExecutor().execute([statement], message, error_message)
    except Exception as e:
        print(f"Error executing statement: {statement}")
        print(f"Error details: {e}")
        break

✅ Creation table success!
✅ Creation table success!
✅ Creation table success!


In [10]:
# Load data to Mysql database
db_config = {
    'mode': 'append',
    'jdbc_url': 'jdbc:mysql://localhost:3306/walmart'
}
connection_properties = {
    'user': 'root',
    'password': '12345',
    'driver': 'com.mysql.cj.jdbc.Driver'
}

customer_table_name = 'customer'
LoadDataToMysql().load_to_db(cleand_customer_df, customer_table_name, db_config, connection_properties)

product_table_name = 'product'
LoadDataToMysql().load_to_db(cleand_product_df, product_table_name, db_config, connection_properties)

purchase_table_name = 'purchase_history_stagging'
LoadDataToMysql().load_to_db(cleand_fas_purchase_df, purchase_table_name, db_config, connection_properties)

In [11]:
'''
    Checking referential integrity: 
        purchase_history và product
        purchase_history và customer
'''
missing_product = cleand_fas_purchase_df.join(
    cleand_product_df,
    cleand_fas_purchase_df.item_purchased == cleand_product_df.item,
    'left_anti'
)
missing_product.show()

+-----------+--------------+---------------+--------+-------------------+-------------+--------------+
|customer_id|item_purchased|purchase_amount|quantity|      date_purchase|review_rating|payment_method|
+-----------+--------------+---------------+--------+-------------------+-------------+--------------+
|       3958|         Tunic|           64.0|       4|2023-06-17 00:00:00|          1.1|          Cash|
|       3962|        Poncho|          184.0|       3|2023-09-25 00:00:00|          2.6|          Cash|
|       3963|        Poncho|          184.0|       2|2023-03-02 00:00:00|          1.6|   Credit Card|
|       3965|        Poncho|          176.0|       1|2023-09-28 00:00:00|          3.6|          Cash|
|       3970|         Tunic|          166.0|       1|2023-02-02 00:00:00|          3.1|   Credit Card|
|       3972|         Tunic|          126.0|       3|2022-11-08 00:00:00|          3.8|   Credit Card|
|       3976|         Tunic|          140.0|       4|2022-12-31 00:00:00|

In [12]:
cleand_customer_df.show()

+-----------+----------+---------+------+-------------+--------------------+------------+-------------------+-------------+------------+
|customer_id|first_name|last_name|gender|date_of_birth|               email|phone_number|        signup_date|      address|        city|
+-----------+----------+---------+------+-------------+--------------------+------------+-------------------+-------------+------------+
|       3964|      Alex|  Johnson|Female|   1972-07-14|robert.williams87...|291-285-3647|2017-09-17 14:04:33|  7255 Oak St|     Phoenix|
|       3970|      Alex|    Jones|Female|   1981-10-08|jane.williams34@m...|869-802-7565|2015-07-10 17:08:42|5623 Maple St|     Chicago|
|       3995|      Alex|    Jones|  Male|   1997-02-05|sarah.martinez133...|924-800-7391|2019-09-13 14:21:15| 5180 Main St|     Houston|
|       4009|      Alex|  Johnson|Female|   1987-09-21|linda.johnson91@m...|178-578-8949|2022-04-10 18:14:58|    45 Oak St|     Houston|
|       4032|      Alex|    Jones|  Male|

In [13]:
missing_customer = cleand_fas_purchase_df.join(
    cleand_customer_df,
    cleand_fas_purchase_df.customer_id == cleand_customer_df.customer_id ,
    'left_anti'
)
missing_customer.show()

+-----------+--------------+---------------+--------+-------------+-------------+--------------+
|customer_id|item_purchased|purchase_amount|quantity|date_purchase|review_rating|payment_method|
+-----------+--------------+---------------+--------+-------------+-------------+--------------+
+-----------+--------------+---------------+--------+-------------+-------------+--------------+



In [14]:

# Create purchase_history table
sql_statement=["""
    CREATE TABLE purchase_history 
    SELECT ph.* 
    FROM purchase_history_stagging AS ph
    JOIN Product AS p 
    ON ph.item_purchased = p.item;
    """,
    
    """
    ALTER TABLE purchase_history
    ADD CONSTRAINT fk_customer_id 
    FOREIGN KEY (customer_id) 
    REFERENCES customer(customer_id) 
    ON DELETE CASCADE;
    """,
    
    """
    ALTER TABLE purchase_history
    ADD CONSTRAINT fk_item_purchased 
    FOREIGN KEY (item_purchased) 
    REFERENCES product(item) 
    ON DELETE CASCADE;   
    """
]

message = '✅ Creation table success!' 
error_message = '❌ Error:'
MysqlExecutor().execute(sql_statement, message, error_message)

✅ Creation table success!
